In [607]:
import numpy as np
import pandas as pd
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [587]:
# Load and format Dataset 1
car_data = pd.read_csv('car.data',header=None)
car_data = car_data.rename(columns={0:'buying',
                                    1:'maint',
                                    2:'doors',
                                    3:'persons',
                                    4:'lug-boot',
                                    5:'safety',
                                    6:'CLASSIFICATION'})

ohc = OneHotEncoder()
ohc.fit(car_data.drop(columns='CLASSIFICATION'))
data = ohc.transform(car_data.drop(columns='CLASSIFICATION')).toarray()
cols = ohc.get_feature_names()
car_data_onehot = pd.DataFrame(data,columns=ohc.get_feature_names())

#Display
display(car_data_onehot.head())

,x0_high,x0_low,x0_med,x0_vhigh,x1_high,x1_low,x1_med,x1_vhigh,x2_2,x2_3,...,x2_5more,x3_2,x3_4,x3_more,x4_big,x4_med,x4_small,x5_high,x5_low,x5_med
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [698]:
# Load and format Dataset 2
chess_data = pd.read_csv('kr-vs-kp.data', delimiter=',',header=None)
chess_data = chess_data.rename(index=str,columns={0:'bkblk',1:'bknwy',2:'bkon8',3:'bkona',4:'bkspr',5:'bkxbq',
                                    6:'bkxcr',7:'bkxwp',8:'blxwp',9:'bxqsq',10:'cntxt',11:'dsopp',
                                    12:'dwipd',13:'hdchk',14:'katri',15:'mulch',16:'qxmsq',17:'r2ar8',
                                    18:'reskd',19:'reskr',20:'rimmx',21:'rkxwp',22:'rxmsq',23:'simpl',
                                    24:'skach',25:'skewr',26:'skrxp',27:'spcop',28:'stlmt',29:'thrsk',
                                    30:'wkcti',31:'wkna8',32:'wknck',33:'wkovl',34:'wkpos',35:'wtoeg',
                                    36:'CLASSIFICATION'})

ohc = OneHotEncoder(handle_unknown='ignore')
ohc.fit(chess_data.drop(columns='CLASSIFICATION'))
data = ohc.transform(chess_data.drop(columns='CLASSIFICATION')).toarray()
cols = ohc.get_feature_names()
chess_df = pd.DataFrame(data,columns=cols)
#Display
chess_data.head()

,bkblk,bknwy,bkon8,bkona,bkspr,bkxbq,bkxcr,bkxwp,blxwp,bxqsq,...,spcop,stlmt,thrsk,wkcti,wkna8,wknck,wkovl,wkpos,wtoeg,CLASSIFICATION
0,f,f,f,f,f,f,f,f,f,f,...,f,f,f,f,f,f,t,t,n,won
1,f,f,f,f,t,f,f,f,f,f,...,f,f,f,f,f,f,t,t,n,won
2,f,f,f,f,t,f,t,f,f,f,...,f,f,f,f,f,f,t,t,n,won
3,f,f,f,f,f,f,f,f,t,f,...,f,f,f,f,f,f,t,t,n,won
4,f,f,f,f,f,f,f,f,f,f,...,f,f,f,f,f,f,t,t,n,won


In [589]:
# Load and format Dataset 3
activity_labels = {1:'WALKING',
                   2:'WALKING_UPSTAIRS',
                   3:'WALKING_DOWNSTAIRS',
                   4:'SITTING',
                   5:'STANDING',
                   6:'LAYING'}

#loading and formatting training
f = open('UCI HAR Dataset/train/X_train.txt', 'r')
f.readline()
lines = f.readlines()
f.close()
activity_arr = []
for ln in range(len(lines)):
    stripped = lines[ln].strip()
    formatted = re.sub('[  ]+', ' ', stripped)
    activity_arr.append(formatted.split(' '))
f = open('UCI HAR Dataset/features.txt')
lines = f.readlines()
f.close()
cols = pd.Series(lines).apply(lambda x: re.sub(re.match('[\d]+[\s]+',x[0:-1]).group(), '', x[0:-1])).values
activity_data_train = pd.DataFrame(activity_arr,columns=cols)
df = pd.read_csv('UCI HAR Dataset/train/y_train.txt')
activity_data_train['CLASSIFICATION'] = df['5']

#loading and formatting testing
f = open('UCI HAR Dataset/test/X_test.txt', 'r')
f.readline()
lines = f.readlines()
f.close()
activity_arr = []
for ln in range(len(lines)):
    stripped = lines[ln].strip()
    formatted = re.sub('[  ]+', ' ', stripped)
    activity_arr.append(formatted.split(' '))
activity_data_test = pd.DataFrame(activity_arr,columns=cols)
df = pd.read_csv('UCI HAR Dataset/test/y_test.txt')
activity_data_test['CLASSIFICATION'] = df['5']
activity_data = pd.concat([activity_data_train,activity_data_test])

#Display
activity_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",CLASSIFICATION
0,2.7841883e-001,-1.6410568e-002,-1.2352019e-001,-9.9824528e-001,-9.7530022e-001,-9.6032199e-001,-9.9880719e-001,-9.7491437e-001,-9.5768622e-001,-9.4306751e-001,...,-5.9505094e-001,-8.6149931e-001,5.3476955e-002,-7.4345661e-003,-7.3262621e-001,7.0351059e-001,-8.4478760e-001,1.8028889e-001,-5.4316717e-002,5
1,2.7965306e-001,-1.9467156e-002,-1.1346169e-001,-9.9537956e-001,-9.6718701e-001,-9.7894396e-001,-9.9651994e-001,-9.6366837e-001,-9.7746859e-001,-9.3869155e-001,...,-3.9074815e-001,-7.6010372e-001,-1.1855926e-001,1.7789948e-001,1.0069921e-001,8.0852908e-001,-8.4893347e-001,1.8063731e-001,-4.9117815e-002,5
2,2.7917394e-001,-2.6200646e-002,-1.2328257e-001,-9.9609149e-001,-9.8340270e-001,-9.9067510e-001,-9.9709947e-001,-9.8274984e-001,-9.8930250e-001,-9.3869155e-001,...,-1.1729020e-001,-4.8284451e-001,-3.6787973e-002,-1.2892494e-002,6.4001104e-001,-4.8536645e-001,-8.4864938e-001,1.8193476e-001,-4.7663183e-002,5
3,2.7662877e-001,-1.6569655e-002,-1.1536185e-001,-9.9813862e-001,-9.8081727e-001,-9.9048163e-001,-9.9832113e-001,-9.7967187e-001,-9.9044113e-001,-9.4246912e-001,...,-3.5147093e-001,-6.9920515e-001,1.2332005e-001,1.2254196e-001,6.9357829e-001,-6.1597061e-001,-8.4786525e-001,1.8515116e-001,-4.3892254e-002,5
4,2.7719877e-001,-1.0097850e-002,-1.0513725e-001,-9.9733496e-001,-9.9048681e-001,-9.9542003e-001,-9.9762740e-001,-9.9021769e-001,-9.9554890e-001,-9.4246912e-001,...,-5.4541011e-001,-8.4461930e-001,8.2632155e-002,-1.4343901e-001,2.7504075e-001,-3.6822404e-001,-8.4963158e-001,1.8482251e-001,-4.2126383e-002,5


In [618]:
trials = 3
split_list = [.2,.5,.8]
test_split = [.8,.5,.2]

neighbor_list = [2,3,4,5,6]

rf_n_estimators = [10,50,100]

max_depths = [5,10,20,None]
criteria = ['gini','entropy']

rf_info_arr = []
knn_info_arr = []
dt_info_arr = []

rf_accs = []
knn_accs = []
dt_accs = []

for split in range(len(split_list)):
    
    rf_train_accs = []
    rf_cv_accs = []
    rf_test_accs = []
    
    knn_train_accs = []
    knn_cv_accs = []
    knn_test_accs = []
    
    dt_train_accs = []
    dt_cv_accs = []
    dt_test_accs = []
    
    for trial in range(trials):
        X_train, X_test, y_train, y_test = train_test_split(car_data_onehot,
                                        car_data['CLASSIFICATION'],
                                        train_size=split_list[split],test_size=test_split[split])

        rfclf = RandomForestClassifier(n_estimators=10)
        rf_params = {'max_depth':max_depths,'criterion':criteria,'n_estimators':rf_n_estimators}
        gridsearch = GridSearchCV(rfclf,rf_params,cv=5)

        results = gridsearch.fit(X_train,y_train).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                     max_depth=best_params['max_depth'],
                                     criterion=best_params['criterion'])
        clf.fit(X_train,y_train)

        rf_train_acc = clf.score(X_train,y_train)
        rf_train_accs.append(rf_train_acc)

        rf_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        rf_cv_accs.append(rf_cv_acc)

        rf_test_acc = clf.score(X_test,y_test)
        rf_test_accs.append(rf_test_acc)

        rf_info_arr.append([trial, split_list[split], rf_train_acc, rf_cv_acc, rf_test_acc])


        knnclf = KNeighborsClassifier()
        knn_params = {'n_neighbors':neighbor_list}
        gridsearch = GridSearchCV(knnclf,knn_params,cv=5)

        results = gridsearch.fit(X_train,y_train).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
        clf.fit(X_train,y_train)

        knn_train_acc = clf.score(X_train,y_train)
        knn_train_accs.append(knn_train_acc)

        knn_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        knn_cv_accs.append(knn_cv_acc)

        knn_test_acc = clf.score(X_test, y_test)
        knn_test_accs.append(knn_test_acc)

        knn_info_arr.append([trial, split_list[split], knn_train_acc, knn_cv_acc, knn_test_acc])


        dtclf = DecisionTreeClassifier()
        dt_params = {'max_depth':max_depths,'criterion':criteria}
        gridsearch = GridSearchCV(dtclf,dt_params,cv=5)

        results = gridsearch.fit(X_train,y_train).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = RandomForestClassifier(max_depth=best_params['max_depth'],
                                     criterion=best_params['criterion'])
        clf.fit(X_train,y_train)

        dt_train_acc = clf.score(X_train,y_train)
        dt_train_accs.append(dt_train_acc)

        dt_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        dt_cv_accs.append(dt_cv_acc)

        dt_test_acc = clf.score(X_test, y_test)
        dt_test_accs.append(dt_test_acc)

        dt_info_arr.append([trial, split_list[split], dt_train_acc, dt_cv_acc, dt_test_acc])

    rf_accs.append(np.mean(rf_train_accs))
    rf_accs.append(np.mean(rf_cv_accs))
    rf_accs.append(np.mean(rf_test_accs))
    
    knn_accs.append(np.mean(knn_train_accs))
    knn_accs.append(np.mean(knn_cv_accs))
    knn_accs.append(np.mean(knn_test_accs))
    
    dt_accs.append(np.mean(dt_train_accs))
    dt_accs.append(np.mean(dt_cv_accs))
    dt_accs.append(np.mean(dt_test_accs))
    
#Detailed Data
d1_rf_df = pd.DataFrame(rf_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])
d1_knn_df = pd.DataFrame(knn_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])
d1_dt_df = pd.DataFrame(dt_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])

#At a glance Data
idx = pd.MultiIndex.from_product([['20%', '50%', '80%'],
                                  ['Train', 'CV', 'Test']],
                                 names=['Train Size', 'Accuracy'])
col = ['Decision Tree', 'KNN', 'Random Forest']
d1_scores = pd.DataFrame('-', idx, col)
d1_scores['Decision Tree'] = dt_accs
d1_scores['KNN'] = knn_accs
d1_scores['Random Forest'] = rf_accs

In [689]:
trials = 3
split_list = [.2,.5,.8]
test_split = [.8,.5,.2]

rf_info_arr = []
knn_info_arr = []
dt_info_arr = []

rf_accs = []
knn_accs = []
dt_accs = []

for split in range(len(split_list)):
    for trial in range(trials):
        X_train, X_test, y_train, y_test = train_test_split(chess_df,
                                        chess_data['CLASSIFICATION'],
                                        train_size=split_list[split],test_size=test_split[split])
        
        rfclf = RandomForestClassifier(n_estimators=10)
        rf_params = {'max_depth':max_depths,'criterion':criteria,'n_estimators':rf_n_estimators}
        gridsearch = GridSearchCV(rfclf,rf_params,cv=5)

        results = gridsearch.fit(X_train,y_train).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                     max_depth=best_params['max_depth'],
                                     criterion=best_params['criterion'])
        clf.fit(X_train,y_train)

        rf_train_acc = clf.score(X_train,y_train)
        rf_train_accs.append(rf_train_acc)

        rf_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        rf_cv_accs.append(rf_cv_acc)

        rf_test_acc = clf.score(X_test,y_test)
        rf_test_accs.append(rf_test_acc)

        rf_info_arr.append([trial, split_list[split], rf_train_acc, rf_cv_acc, rf_test_acc])


        knnclf = KNeighborsClassifier()
        knn_params = {'n_neighbors':neighbor_list}
        gridsearch = GridSearchCV(knnclf,knn_params,cv=5)

        results = gridsearch.fit(X_train,y_train).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
        clf.fit(X_train,y_train)

        knn_train_acc = clf.score(X_train,y_train)
        knn_train_accs.append(knn_train_acc)

        knn_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        knn_cv_accs.append(knn_cv_acc)

        knn_test_acc = clf.score(X_test, y_test)
        knn_test_accs.append(knn_test_acc)

        knn_info_arr.append([trial, split_list[split], knn_train_acc, knn_cv_acc, knn_test_acc])


        dtclf = DecisionTreeClassifier()
        dt_params = {'max_depth':max_depths,'criterion':criteria}
        gridsearch = GridSearchCV(dtclf,dt_params,cv=5)

        results = gridsearch.fit(X_train,y_train).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = RandomForestClassifier(max_depth=best_params['max_depth'],
                                     criterion=best_params['criterion'])
        clf.fit(X_train,y_train)

        dt_train_acc = clf.score(X_train,y_train)
        dt_train_accs.append(dt_train_acc)

        dt_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        dt_cv_accs.append(dt_cv_acc)

        dt_test_acc = clf.score(X_test, y_test)
        dt_test_accs.append(dt_test_acc)

        dt_info_arr.append([trial, split_list[split], dt_train_acc, dt_cv_acc, dt_test_acc])
    
    rf_accs.append(np.mean(rf_train_accs))
    rf_accs.append(np.mean(rf_cv_accs))
    rf_accs.append(np.mean(rf_test_accs))
    
    knn_accs.append(np.mean(knn_train_accs))
    knn_accs.append(np.mean(knn_cv_accs))
    knn_accs.append(np.mean(knn_test_accs))
    
    dt_accs.append(np.mean(dt_train_accs))
    dt_accs.append(np.mean(dt_cv_accs))
    dt_accs.append(np.mean(dt_test_accs))
    
#Detailed Data
d2_rf_df = pd.DataFrame(rf_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])
d2_knn_df = pd.DataFrame(knn_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])
d2_dt_df = pd.DataFrame(dt_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])

#At a glance Data
idx = pd.MultiIndex.from_product([['20%', '50%', '80%'],
                                  ['Train', 'CV', 'Test']],
                                 names=['Train Size', 'Accuracy'])
col = ['Decision Tree', 'KNN', 'Random Forest']
d2_scores = pd.DataFrame('-', idx, col)
d2_scores['Decision Tree'] = dt_accs
d2_scores['KNN'] = knn_accs
d2_scores['Random Forest'] = rf_accs

In [648]:
trials = 3
split_list = [.2,.5,.8]
test_split = [.8,.5,.2]

rf_info_arr = []
knn_info_arr = []
dt_info_arr = []

rf_accs = []
knn_accs = []
dt_accs = []

for split in range(len(split_list)):
    for trial in range(trials):
        X_train, X_test, y_train, y_test = train_test_split(activity_data.drop(columns='CLASSIFICATION'),
                                        activity_data['CLASSIFICATION'],
                                        train_size=split_list[split],test_size=test_split[split])
        
        train_samp = X_train[0:500]
        train_targ = y_train[0:500]
        
        rfclf = RandomForestClassifier(n_estimators=10)
        rf_params = {'max_depth':max_depths,'criterion':criteria,'n_estimators':rf_n_estimators}
        gridsearch = GridSearchCV(rfclf,rf_params,cv=5)

        results = gridsearch.fit(train_samp,train_targ).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                     max_depth=best_params['max_depth'],
                                     criterion=best_params['criterion'])
        clf.fit(X_train,y_train)

        rf_train_acc = clf.score(X_train,y_train)
        rf_train_accs.append(rf_train_acc)

        rf_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        rf_cv_accs.append(rf_cv_acc)

        rf_test_acc = clf.score(X_test,y_test)
        rf_test_accs.append(rf_test_acc)

        rf_info_arr.append([trial, split_list[split], rf_train_acc, rf_cv_acc, rf_test_acc])


        knnclf = KNeighborsClassifier()
        knn_params = {'n_neighbors':neighbor_list}
        gridsearch = GridSearchCV(knnclf,knn_params,cv=5)

        results = gridsearch.fit(train_samp,train_targ).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
        clf.fit(X_train,y_train)

        knn_train_acc = clf.score(X_train,y_train)
        knn_train_accs.append(knn_train_acc)

        knn_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        knn_cv_accs.append(knn_cv_acc)

        knn_test_acc = clf.score(X_test, y_test)
        knn_test_accs.append(knn_test_acc)

        knn_info_arr.append([trial, split_list[split], knn_train_acc, knn_cv_acc, knn_test_acc])


        dtclf = DecisionTreeClassifier()
        dt_params = {'max_depth':max_depths,'criterion':criteria}
        gridsearch = GridSearchCV(dtclf,dt_params,cv=5)

        results = gridsearch.fit(train_samp,train_targ).cv_results_
        best_param_idx = pd.Series(results['rank_test_score']).idxmin()
        best_params = results['params'][best_param_idx]

        clf = RandomForestClassifier(max_depth=best_params['max_depth'],
                                     criterion=best_params['criterion'])
        clf.fit(X_train,y_train)

        dt_train_acc = clf.score(X_train,y_train)
        dt_train_accs.append(dt_train_acc)

        dt_cv_acc = cross_val_score(clf, X_train,y_train,cv=3).mean()
        dt_cv_accs.append(dt_cv_acc)

        dt_test_acc = clf.score(X_test, y_test)
        dt_test_accs.append(dt_test_acc)

        dt_info_arr.append([trial, split_list[split], dt_train_acc, dt_cv_acc, dt_test_acc])
    
    rf_accs.append(np.mean(rf_train_accs))
    rf_accs.append(np.mean(rf_cv_accs))
    rf_accs.append(np.mean(rf_test_accs))
    
    knn_accs.append(np.mean(knn_train_accs))
    knn_accs.append(np.mean(knn_cv_accs))
    knn_accs.append(np.mean(knn_test_accs))
    
    dt_accs.append(np.mean(dt_train_accs))
    dt_accs.append(np.mean(dt_cv_accs))
    dt_accs.append(np.mean(dt_test_accs))
    
#Detailed Data
d3_rf_df = pd.DataFrame(rf_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])
d3_knn_df = pd.DataFrame(knn_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])
d3_dt_df = pd.DataFrame(dt_info_arr,columns=['Trial','Train Split', 'Train Acc', 'CV Acc', 'Test Acc'])

#At a glance Data
idx = pd.MultiIndex.from_product([['20%', '50%', '80%'],
                                  ['Train', 'CV', 'Test']],
                                 names=['Train Size', 'Accuracy'])
col = ['Decision Tree', 'KNN', 'Random Forest']
d3_scores = pd.DataFrame('-', idx, col)
d3_scores['Decision Tree'] = dt_accs
d3_scores['KNN'] = knn_accs
d3_scores['Random Forest'] = rf_accs

In [691]:
split_labels = ['20%', '50%', '80%']
class_labels = ['Decision Tree', 'KNN', 'Random Forest']
acc_labels = ['Train', 'CV', 'Test']
df_list = [d1_scores, d2_scores, d3_scores]

scores = {}
for split in split_labels:
    scores[split] = {}
    for acc in acc_labels:
        scores[split][acc] = {}
        for classifier in class_labels:
            scores[split][acc][classifier] = []
            for df in df_list:
                scores[split][acc][classifier].append(df[classifier][split][acc])

small_train = [np.mean(scores['20%']['Train']['Decision Tree']),
               np.mean(scores['20%']['Train']['KNN']),
               np.mean(scores['20%']['Train']['Random Forest'])]

small_cv = [np.mean(scores['20%']['CV']['Decision Tree']),
            np.mean(scores['20%']['CV']['KNN']),
            np.mean(scores['20%']['CV']['Random Forest'])]

small_test = [np.mean(scores['20%']['Test']['Decision Tree']),
              np.mean(scores['20%']['Test']['KNN']),
              np.mean(scores['20%']['Test']['Random Forest'])]

med_train = [np.mean(scores['50%']['Train']['Decision Tree']),
             np.mean(scores['50%']['Train']['KNN']),
             np.mean(scores['50%']['Train']['Random Forest'])]

med_cv = [np.mean(scores['50%']['CV']['Decision Tree']),
          np.mean(scores['50%']['CV']['KNN']),
          np.mean(scores['50%']['CV']['Random Forest'])]

med_test = [np.mean(scores['50%']['Test']['Decision Tree']),
            np.mean(scores['50%']['Test']['KNN']),
            np.mean(scores['50%']['Test']['Random Forest'])]

large_train = [np.mean(scores['80%']['Train']['Decision Tree']),
               np.mean(scores['80%']['Train']['KNN']),
               np.mean(scores['80%']['Train']['Random Forest'])]

large_cv = [np.mean(scores['80%']['CV']['Decision Tree']),
            np.mean(scores['80%']['CV']['KNN']),
            np.mean(scores['80%']['CV']['Random Forest'])]

large_test = [np.mean(scores['80%']['Test']['Decision Tree']),
              np.mean(scores['80%']['Test']['KNN']),
              np.mean(scores['80%']['Test']['Random Forest'])]

small_df = pd.DataFrame([small_train,small_cv,small_test],
             index=['Train','CV','Test'],
             columns=['Decision Tree','KNN','Random Forest'])

med_df = pd.DataFrame([med_train,med_cv,med_test],
             index=['Train','CV','Test'],
             columns=['Decision Tree','KNN','Random Forest'])

large_df = pd.DataFrame([large_train,large_cv,large_test],
             index=['Train','CV','Test'],
             columns=['Decision Tree','KNN','Random Forest'])
print('20% Split Average Accuracy Across Datasets:')
display(small_df)

print('50% Split Average Accuracy Across Datasets:')
display(med_df)

print('80% Split Average Accuracy Across Datasets:')
display(large_df)

20% Split Average Accuracy Across Datasets:


,Decision Tree,KNN,Random Forest
Train,0.960287,0.930458,0.998534
CV,0.892223,0.878002,0.914041
Test,0.901554,0.879479,0.918994


50% Split Average Accuracy Across Datasets:


,Decision Tree,KNN,Random Forest
Train,0.964571,0.940279,0.998648
CV,0.913268,0.893287,0.933333
Test,0.919715,0.902638,0.941695


80% Split Average Accuracy Across Datasets:


,Decision Tree,KNN,Random Forest
Train,0.968871,0.945847,0.998474
CV,0.930618,0.907826,0.945602
Test,0.930988,0.909760,0.948565


In [692]:
dt_avg_train = np.mean([small_df['Decision Tree'].loc['Train'],
                        med_df['Decision Tree'].loc['Train'],
                        large_df['Decision Tree'].loc['Train']])

dt_avg_cv = np.mean([small_df['Decision Tree'].loc['CV'],
                     med_df['Decision Tree'].loc['CV'],
                     large_df['Decision Tree'].loc['CV']])

dt_avg_test = np.mean([small_df['Decision Tree'].loc['Test'],
                       med_df['Decision Tree'].loc['Test'],
                       large_df['Decision Tree'].loc['Test']])



knn_avg_train = np.mean([small_df['KNN'].loc['Train'],
                        med_df['KNN'].loc['Train'],
                        large_df['KNN'].loc['Train']])

knn_avg_cv = np.mean([small_df['KNN'].loc['CV'],
                      med_df['KNN'].loc['CV'],
                      large_df['KNN'].loc['CV']])

knn_avg_test = np.mean([small_df['KNN'].loc['Test'],
                       med_df['KNN'].loc['Test'],
                       large_df['KNN'].loc['Test']])



rf_avg_train = np.mean([small_df['Random Forest'].loc['Train'],
                        med_df['Random Forest'].loc['Train'],
                        large_df['Random Forest'].loc['Train']])

rf_avg_cv = np.mean([small_df['Random Forest'].loc['CV'],
                     med_df['Random Forest'].loc['CV'],
                     large_df['Random Forest'].loc['CV']])

rf_avg_test = np.mean([small_df['Random Forest'].loc['Test'],
                       med_df['Random Forest'].loc['Test'],
                       large_df['Random Forest'].loc['Test']])

train_final_avg = [dt_avg_train,knn_avg_train,rf_avg_train]
cv_final_avg = [dt_avg_cv,knn_avg_cv,rf_avg_cv]
test_final_avg = [dt_avg_test,knn_avg_test,rf_avg_test]

final_df = pd.DataFrame([train_final_avg,cv_final_avg,test_final_avg],
                         index=['Train','CV','Test'],
                         columns=['Decision Tree', 'KNN', 'Random Forest'])
print('Average Accuracy Across All Splits and All Datasets:')
final_df

Average Accuracy Across All Splits and All Datasets:


,Decision Tree,KNN,Random Forest
Train,0.964576,0.938861,0.998552
CV,0.912036,0.893039,0.930992
Test,0.917419,0.897292,0.936418


In [693]:
print('Dataset 1:')
display(d1_scores)
print('Dataset 2:')
display(d2_scores)
print('Dataset 3:')
d3_scores

Dataset 1:


Decision Tree       KNN  Random Forest
Train Size Accuracy                                        
20%        Train          0.996135  0.923671       1.000000
           CV             0.846212  0.831903       0.882152
           Test           0.867920  0.829356       0.889130
50%        Train          0.996914  0.941358       1.000000
           CV             0.897785  0.867325       0.927875
           Test           0.910880  0.886960       0.944830
80%        Train          0.998312  0.947178       1.000000
           CV             0.938251  0.900617       0.954902
           Test           0.932563  0.895954       0.955684

Dataset 2:


Decision Tree       KNN  Random Forest
Train Size Accuracy                                        
20%        Train          0.952940  0.941673       0.997644
           CV             0.923657  0.907320       0.938161
           Test           0.927607  0.911597       0.942371
50%        Train          0.958146  0.944886       0.997906
           CV             0.929614  0.908463       0.943108
           Test           0.933588  0.913678       0.947199
80%        Train          0.961862  0.948676       0.998115
           CV             0.934591  0.911200       0.947362
           Test           0.938666  0.917727       0.951645

Dataset 3:


Decision Tree       KNN  Random Forest
Train Size Accuracy                                        
20%        Train          0.931785  0.926030       0.997958
           CV             0.906800  0.894783       0.921812
           Test           0.909136  0.897483       0.925481
50%        Train          0.938654  0.934592       0.998039
           CV             0.912405  0.904075       0.929015
           Test           0.914677  0.907275       0.933056
80%        Train          0.946441  0.941687       0.997308
           CV             0.919010  0.911662       0.934543
           Test           0.921734  0.915598       0.938366

In [694]:
print('RANDOM FOREST:')
print('dataset 1:')
display(d1_rf_df)
print('dataset 2:')
display(d2_rf_df)
print('dataset 3:')
display(d3_rf_df)
print('KNN:')
print('dataset 1:')
display(d1_knn_df)
print('dataset 2:')
display(d2_knn_df)
print('dataset 3:')
display(d3_knn_df)
print('DECISION TREE:')
print('dataset 1')
display(d1_dt_df)
print('dataset 2')
display(d2_dt_df)
print('dataset 3')
display(d3_dt_df)

RANDOM FOREST:
dataset 1:


,Trial,Train Split,Train Acc,CV Acc,Test Acc
0,0,0.2,1.0,0.872332,0.890817
1,1,0.2,1.0,0.883989,0.892263
2,2,0.2,1.0,0.890135,0.884309
3,0,0.5,1.0,0.934066,0.951389
4,1,0.5,1.0,0.923615,0.959491
5,2,0.5,1.0,0.925945,0.923611
6,0,0.8,1.0,0.950803,0.968208
7,1,0.8,1.0,0.958048,0.930636
8,2,0.8,1.0,0.955855,0.968208


dataset 2:


,Trial,Train Split,Train Acc,CV Acc,Test Acc
0,0,0.2,1.0,0.954609,0.975362
1,1,0.2,1.0,0.962441,0.971842
2,2,0.2,1.0,0.973403,0.964020
3,0,0.5,1.0,0.983101,0.984355
4,1,0.5,1.0,0.981235,0.991239
5,2,0.5,1.0,0.983723,0.981852
6,0,0.8,1.0,0.984345,0.993750
7,1,0.8,1.0,0.986303,0.992188
8,2,0.8,1.0,0.986306,0.989062


dataset 3:


...

,Trial,Train Split,Train Acc,CV Acc,Test Acc
0,0,0.2,0.997101,0.843270,0.865510
1,1,0.2,0.994203,0.843226,0.877802
2,2,0.2,0.997101,0.852142,0.860448
3,0,0.5,0.997685,0.908586,0.917824
4,1,0.5,0.994213,0.886593,0.910880
5,2,0.5,0.998843,0.898175,0.903935
6,0,0.8,0.998553,0.929081,0.950867
7,1,0.8,0.996382,0.952236,0.910405
8,2,0.8,1.000000,0.933436,0.936416


dataset 2


,Trial,Train Split,Train Acc,CV Acc,Test Acc
0,0,0.2,1.000000,0.956196,0.971842
1,1,0.2,0.996870,0.942097,0.969887
2,2,0.2,0.998435,0.970266,0.964411
3,0,0.5,1.000000,0.981224,0.977472
4,1,0.5,1.000000,0.975599,0.985607
5,2,0.5,0.999374,0.974963,0.981227
6,0,0.8,0.999609,0.981996,0.990625
7,1,0.8,0.999218,0.985133,0.987500
8,2,0.8,0.987089,0.971047,0.975000


dataset 3


,Trial,Train Split,Train Acc,CV Acc,Test Acc
0,0,0.2,0.937834,0.912534,0.904588
1,1,0.2,0.999029,0.924256,0.945011
2,2,0.2,0.998057,0.927636,0.940884
3,0,0.5,0.925602,0.918797,0.902700
4,1,0.5,0.998640,0.954740,0.964265
5,2,0.5,0.994755,0.947748,0.960186
6,0,0.8,0.987010,0.954230,0.960194
7,1,0.8,0.993808,0.961516,0.971845
8,2,0.8,0.998665,0.960183,0.960194
